In [ ]:
import itertools
import time
import matplotlib.pyplot as plt
import numpy as np
import sys
import pandas as pd
from tqdm import tqdm
from random import randint, choice, uniform
from math import ceil
from statistics import stdev, mean
from qiskit import *
from qiskit.visualization import plot_histogram
from IPython.core.display import display, HTML
import json
import numpy as np
import random

In [ ]:
class Graph():
    def __init__(self, N, randomize=True):
        ''' Initialize a random graph with N vertices. '''
        self.N = N
        self.E = 0
        self.adj = {n:dict() for n in range(N)}

        # For storing information about each run.
        self.currentScore = float('-inf')
        self.currentBest = ""
        self.runs = []

        # Randomly generate edges
        if randomize:
            self.randomize()

    def randomize(self):
        ''' Randomly generate edges for this graph. '''

        # Generate list of tuples for all possible directed edges.
        all_possible_edges = set([(x,y) for x in range(self.N) for y in range(self.N) if x != y])

        # Sanity check, ensuring we generated the correct number of edges.
        e_gen = len(all_possible_edges) / 2
        e_shd = self.N * (self.N-1) / 2
        assert e_gen == e_shd , "%d != %d" % (e_gen, e_shd)

        # Choose a random number of edges for this graph to have. 
        # Note, we stop at len/2 because we generated directed edges,
        # so each edge counts twice.
        num_edges = len(all_possible_edges)//2
        for i in range(num_edges):
            # Choose an edge, remove it and its directed complement from the list.
            e = choice(list(all_possible_edges))
            all_possible_edges.remove(e)
            all_possible_edges.remove(e[::-1])

            # Unpack tuple into vertex ints.
            u, v = int(e[0]), int(e[1])

            # Choose a random weight for each edge.
            weight = randint(1, 100)

            #weight = 1
            self.add_edge(u, v, weight)


    def add_edge(self, u, v, weight):
        ''' Add an edge to the graph. '''
        self.E += 1
        self.adj[u][v] = weight

    def get_edges(self):
        ''' Get a list of all edges. '''
        edges = []
        for u in self.adj:
            for v in self.adj[u]:
                edges.append((u, v, self.adj[u][v]))
        return edges

    def get_score(self,bitstring):
        ''' Score a candidate solution. '''
        assert len(bitstring) == self.N

        score = 0

        # For every edge u,v in the graph, add the weight
        # of the edge if u,v belong to different cuts
        # given this canddiate solution.

        for u in self.adj:
            for v in self.adj[u]:
                if bitstring[u] != bitstring[v]:
                    score += self.adj[u][v]
        return score

    def optimal_score(self):
        '''
        Returns (score, solutions) holding the best possible solution to the
        MaxCut problem with this graph.
        '''

        best = 0
        best_val = []

        # Iterate over all possible candidate bitstrings
        # Note: the bitstrings from 0 - N/2 are symmetrically
        # equivalent to those above
        for i in range(ceil((2 ** self.N)/2)):
            # Convert number to 0-padded bitstring.
            bitstring = bin(i)[2:]
            bitstring = (self.N - len(bitstring)) * "0" + bitstring

            sc = self.get_score(bitstring)
            if sc > best:
                best = sc
                best_val = [bitstring]
            elif sc == best:
                best_val.append(bitstring)
        return best, best_val

    def edges_cut(self, bitstring):
        ''' Given a candidate solution, return the number of edges that this solution cuts. '''
        num = 0
        for u in self.adj:
            for v in self.adj[u]:
                if bitstring[u] != bitstring[v]:
                    num += 1
        return num

    def update_score(self, bitstring):
        ''' Scores the given bitstring and keeps track of best. '''
        score = self.get_score(bitstring)
        if score > self.currentScore:
            self.currentScore = score
            self.currentBest = bitstring
        return score
    
    def clear_runs(self):
        ''' Clear data from past runs. '''
        self.currentScore = float('-inf')
        self.currentBest = ""
        self.runs = []
        
    def add_run(self, gamma, beta, expected_value):
        ''' Save the data from each run iteration. '''
        self.runs.append([gamma, beta, expected_value])
        
    def __str__(self):
        return "Graph with %d vertices %d edges.\nAdjacency List: %s" % (self.N, self.E, self.adj)

In [ ]:
count = 0
meanAcc = 0.0
def get_expectation(x,n):
    global count
    global meanAcc
    g = Graph(n)
    beta, gamma = x
    q = QuantumRegister(g.N)
    c = ClassicalRegister(g.N)
    qc = QuantumCircuit(q, c)
    for i in range(g.N):
        qc.h(q[i])
    for edge in g.get_edges():
        u, v, w = edge
        qc.cx(q[u], q[v])
        qc.u1(gamma*w, q[v])
        qc.cx(q[u], q[v])
    for i in range(g.N):
        qc.h(q[i])
        qc.u1(-2*beta, q[i])
        qc.h(q[i])
    for i in range(g.N):
        qc.measure(q[i], c[i])
    backend = BasicAer.get_backend("qasm_simulator")
    job = execute(qc, backend, shots=8192)
    results = job.result()
    result_dict = results.get_counts(qc)
    exp = 0
    for bitstring in result_dict:
        prob = np.float(result_dict[bitstring]) / 8192
        score = g.update_score(bitstring)
        exp += score * prob
    # Calculate the expected value of the candidate bitstrings.
    v=list(result_dict.values())
    k=list(result_dict.keys())
    print(g.optimal_score()[0])
    print(exp)
    print(float(exp/g.optimal_score()[0]))
    meanAcc+=float(exp/g.optimal_score()[0])
    print("----------")
    if(g.get_score(k[v.index(max(v))])==g.optimal_score()[0]):
        count+=1
params = [5.711986642890533,0.8250647373064104]
for i in range(100):
    n = random.randint(5,15)
    get_expectation(params,n)
print(meanAcc)
print("--------------------")
print("Correct Answer Count by Approach 1 : " + str(count))
    

In [ ]:
count = 0
meanAcc = 0.0
pDev = list()
nDev = list()
def get_expectation(x,n,val):
    global pDev
    global nDev
    global count
    global meanAcc
    g = Graph(n)
    beta, gamma = x
    q = QuantumRegister(g.N)
    c = ClassicalRegister(g.N)
    qc = QuantumCircuit(q, c)
    for i in range(g.N):
        qc.h(q[i])
    for edge in g.get_edges():
        u, v, w = edge
        qc.cx(q[u], q[v])
        qc.u1(gamma*w, q[v])
        qc.cx(q[u], q[v])
    for i in range(g.N):
        qc.h(q[i])
        qc.u1(-2*beta, q[i])
        qc.h(q[i])
    for i in range(g.N):
        qc.measure(q[i], c[i])
    backend = BasicAer.get_backend("qasm_simulator")
    job = execute(qc, backend, shots=8192)
    results = job.result()
    result_dict = results.get_counts(qc)
    exp = 0
    for bitstring in result_dict:
        prob = np.float(result_dict[bitstring]) / 8192
        score = g.update_score(bitstring)
        exp += score * prob
    # Calculate the expected value of the candidate bitstrings.
    v=list(result_dict.values())
    k=list(result_dict.keys())
    print(g.optimal_score()[0])
    print(exp*val)
    o = (float((g.optimal_score()[0]-exp*val)/g.optimal_score()[0])*100)
    if o>=0:
        pDev.append(o)
    else:
        nDev.append(o)
    print(o)
    print("----------")
    if(g.get_score(k[v.index(max(v))])==g.optimal_score()[0]):
        count+=1
params = [5.711986642890533,0.8250647373064104]
factor = 1/(0.7477343509947454)
for i in range(100):
    n = random.randint(5,15)
    get_expectation(params,n,factor)
print(pDev)

In [ ]:
global pDev
global nDev
print("Accuracy Metrics of Algorithm")
print(np.percentile(pDev,95))
print(np.percentile(nDev,5))

In [ ]:
count = 0
def get_expectation(x,n,val,a,b):
    global count
    global pDev
    global nDev
    global count
    global meanAcc
    g = Graph(n)
    beta, gamma = x
    q = QuantumRegister(g.N)
    c = ClassicalRegister(g.N)
    qc = QuantumCircuit(q, c)
    for i in range(g.N):
        qc.h(q[i])
    for edge in g.get_edges():
        u, v, w = edge
        qc.cx(q[u], q[v])
        qc.u1(gamma*w, q[v])
        qc.cx(q[u], q[v])
    for i in range(g.N):
        qc.h(q[i])
        qc.u1(-2*beta, q[i])
        qc.h(q[i])
    for i in range(g.N):
        qc.measure(q[i], c[i])
    backend = BasicAer.get_backend("qasm_simulator")
    job = execute(qc, backend, shots=8192)
    results = job.result()
    result_dict = results.get_counts(qc)
    exp = 0
    for bitstring in result_dict:
        prob = np.float(result_dict[bitstring]) / 8192
        score = g.update_score(bitstring)
        exp += score * prob
    # Calculate the expected value of the candidate bitstrings.
    v=list(result_dict.values())
    k=list(result_dict.keys())
    if (exp*val*a<=g.optimal_score()[0] and g.optimal_score()[0]<=(exp*val*b)):
        print("-------------")
        print("True")
        print(g.optimal_score()[0])
        print(exp*val*a)
        print(exp*val*b)
        count+=1
    else:
        print("-------------")
        print("False")
        print(g.optimal_score()[0])
        print(exp*val*a)
        print(exp*val*b)
params = [5.711986642890533,0.8250647373064104]
factor = 1/(0.7444540376466091)
for i in range(100):
    n = random.randint(10,16)
    get_expectation(params,n,factor,0.84496882145,1.0829449852)
print("----------")
print("Correct Predictions : " +str(count) +"/100")

In [4]:
count = 0
ans = 0
import random
def get_expectation(x,n,val): 
    global ans
    global count
    global pDev
    global nDev
    global count
    global meanAcc
    g = Graph(n)
    beta, gamma = x
    q = QuantumRegister(g.N)
    c = ClassicalRegister(g.N)
    qc = QuantumCircuit(q, c)
    for i in range(g.N):
        qc.h(q[i])
    for edge in g.get_edges():
        u, v, w = edge
        qc.cx(q[u], q[v])
        qc.u1(gamma*w, q[v])
        qc.cx(q[u], q[v])
    for i in range(g.N):
        qc.h(q[i])
        qc.u1(-2*beta, q[i])
        qc.h(q[i])
    for i in range(g.N):
        qc.measure(q[i], c[i])
    backend = BasicAer.get_backend("qasm_simulator")
    job = execute(qc, backend, shots=8192)
    results = job.result()
    result_dict = results.get_counts(qc)
    exp = 0
    for bitstring in result_dict:
        prob = np.float(result_dict[bitstring]) / 8192
        score = g.update_score(bitstring)
        exp += score * prob
    # Calculate the expected value of the candidate bitstrings.
    v=list(result_dict.values())
    k=list(result_dict.keys())
    '''
    print(g.optimal_score()[0])
    print(exp*val*0.92427184466)
    print(exp*val*0.92427184466/g.optimal_score()[0])
    '''
    if((exp*val*0.917475/g.optimal_score()[0])>1.0099999):
        print("Violation")
    ans=ans + exp*val*0.917475 /g.optimal_score()[0]
    print("Predicted Value :" + str(exp*val*0.917475))
    print("Actual Value : " + str(g.optimal_score()[0]))
    print("Accuracy : " + str(exp*val*0.917475*100/g.optimal_score()[0])+"%")
params = [5.711986642890533,0.8250647373064104]
factor = 1/(0.7444540376466091)
for i in range(50):
    n = random.randint(10,16)
    get_expectation(params,n,factor)
print("----------------")
print("Mean Accuracy " + str(ans*2)+" %")

Predicted Value :2225.3012496524534
Actual Value : 2392
Accuracy : 93.03098869784505%
Predicted Value :1939.1852067371215
Actual Value : 2081
Accuracy : 93.18525741168291%
Predicted Value :1870.2771802823584
Actual Value : 1974
Accuracy : 94.74555117945077%
Predicted Value :2647.31808024983
Actual Value : 2794
Accuracy : 94.75011024516212%
Predicted Value :2863.41674428031
Actual Value : 2890
Accuracy : 99.08016416194845%
Predicted Value :2103.5599752223566
Actual Value : 2172
Accuracy : 96.84898596788013%
Predicted Value :2231.672278410493
Actual Value : 2329
Accuracy : 95.82105102664204%
Predicted Value :2134.286058802657
Actual Value : 2218
Accuracy : 96.22570147892954%
Predicted Value :1770.628026140567
Actual Value : 1774
Accuracy : 99.80992255583806%
Predicted Value :2489.4822327537145
Actual Value : 2554
Accuracy : 97.47385406240072%
Predicted Value :2078.046072858875
Actual Value : 2166
Accuracy : 95.93933854380771%
Predicted Value :1350.4480809746215
Actual Value : 1496
Accura